# Choice of location for the oil well

You have been provided with oil samples in three regions: in each of 10,000 fields, where the quality of oil and the volume of its reserves have been measured. Build a machine learning model to help determine the region where mining will bring the most profit. Analyze possible profits and risks using the *Bootstrap.* technique

Steps to choose a location:

- In the selected region, they are looking for deposits, for each, the values ​​of the signs are determined;
- Build a model and estimate the volume of reserves;
- Select the deposits with the highest value estimates. The number of fields depends on the company's budget and the cost of developing one well;
- The profit is equal to the total profit of the selected fields.

## Loading and preparing data

### Import the necessary libraries

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy import sqrt

# and fix static variables
state = np.random.RandomState(12345)
scaler = StandardScaler()
model = LinearRegression()

import warnings
warnings.filterwarnings('ignore')

### Load and examine data

In [2]:
data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')

In [3]:
print('data_0:', 
      data_0)
print()
print('data_1:', 
      data_1)
print()
print('data_2:', 
      data_2)

data_0:           id        f0        f1        f2     product
0      txEyH  0.705745 -0.497823  1.221170  105.280062
1      2acmU  1.334711 -0.340164  4.365080   73.037750
2      409Wp  1.022732  0.151990  1.419926   85.265647
3      iJLyR -0.032172  0.139033  2.978566  168.620776
4      Xdl7t  1.988431  0.155413  4.751769  154.036647
...      ...       ...       ...       ...         ...
99995  DLsed  0.971957  0.370953  6.075346  110.744026
99996  QKivN  1.392429 -0.382606  1.273912  122.346843
99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
99998  7kl59  0.998163 -0.528582  1.583869   74.040764
99999  1CWhH  1.764754 -0.266417  5.722849  149.633246

[100000 rows x 5 columns]

data_1:           id         f0         f1        f2     product
0      kBEdx -15.001348  -8.276000 -0.005876    3.179103
1      62mP7  14.272088  -3.475083  0.999183   26.953261
2      vyE1P   6.263187  -5.948386  5.001160  134.766305
3      KcrkZ -13.081196 -11.506057  4.999415  137.945408
4      AHL4

Data Description:
- id - unique identifier of the well;
- f0, f1, f2 - three signs of points (it doesn't matter what they mean, but the signs themselves are significant);
- product - the volume of reserves in the well (thousand barrels).


In [4]:
print('data_0 info:', data_0.info())
print()
print('data_1 info:', data_1.info())
print()
print('data_2 info:', data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
data_0 info: None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
data_1 info: None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (tot

In [5]:
print('data_0 describe:', data_0.describe())
print()
print('data_1 describe:', data_1.describe())
print()
print('data_2 describe:', data_2.describe())
print()

data_0 describe:                   f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

data_1 describe:                   f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985      

### Check the data for duplicates

In [6]:
data_0.duplicated().sum()

0

In [7]:
data_1.duplicated().sum()

0

In [8]:
data_2.duplicated().sum()

0

### Result of examining the data

1. There are no missing values
2. Columns are named correctly, renaming is not required
3. The "id" column is just names and is useless for model training
4. Learning features (features): f0, f1, f2
5. Target attribute (target): product
6. No duplicates

## Train and validate the model

### Data preparation

In [9]:
features_0 = data_0[['f0','f1','f2']]
target_0 = data_0[['product']]

In [10]:
features_1 = data_1[['f0','f1','f2']]
target_1 = data_1[['product']]

In [11]:
features_2 = data_2[['f0','f1','f2']]
target_2 = data_2[['product']]

### Create training/validation sets

#### Sets for the first region

In [12]:
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(features_0, target_0, 
                                                                                      train_size = 0.75, 
                                                                                      random_state = state, 
                                                                                      shuffle = False)
print('features_0_train:', features_0_train.shape)
print('features_0_valid:', features_0_valid.shape)
print('target_0_train:', target_0_train.shape)
print('target_0_valid:', target_0_valid.shape)

features_0_train: (75000, 3)
features_0_valid: (25000, 3)
target_0_train: (75000, 1)
target_0_valid: (25000, 1)


#### Sets for the second region

In [13]:
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(features_1, target_1, 
                                                                                      train_size = 0.75, 
                                                                                      random_state = state, 
                                                                                      shuffle = False)
print('features_1_train:', features_1_train.shape)
print('features_1_valid:', features_1_valid.shape)
print('target_1_train:', target_1_train.shape)
print('target_1_valid:', target_1_valid.shape)

features_1_train: (75000, 3)
features_1_valid: (25000, 3)
target_1_train: (75000, 1)
target_1_valid: (25000, 1)


#### Sets for the third region

In [14]:
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(features_2, target_2, 
                                                                                      train_size = 0.75, 
                                                                                      random_state = state, 
                                                                                      shuffle = False)
print('features_2_train:', features_2_train.shape)
print('features_2_valid:', features_2_valid.shape)
print('target_2_train:', target_2_train.shape)
print('target_2_valid:', target_2_valid.shape)

features_2_train: (75000, 3)
features_2_valid: (25000, 3)
target_2_train: (75000, 1)
target_2_valid: (25000, 1)


## Model training

### Train the model

#### Model for the first region

In [15]:
scaler.fit(features_0_train)

features_0_train = scaler.transform(features_0_train)
features_0_valid = scaler.transform(features_0_valid)

model.fit(features_0_train, target_0_train)

predicted_0_valid = model.predict(features_0_valid)
score_0 = model.score(features_0_valid, target_0_valid)
RMSE_0 = sqrt(mean_squared_error(target_0_valid, predicted_0_valid))


#### Model for the second region

In [16]:
scaler.fit(features_1_train)

features_1_train = scaler.transform(features_1_train)
features_1_valid = scaler.transform(features_1_valid)

model.fit(features_1_train, target_1_train)

predicted_1_valid = model.predict(features_1_valid)
score_1 = model.score(features_1_valid, target_1_valid)
RMSE_1 = sqrt(mean_squared_error(target_1_valid, predicted_1_valid))


#### Model for the third region

In [17]:
scaler.fit(features_2_train)

features_2_train = scaler.transform(features_2_train)
features_2_valid = scaler.transform(features_2_valid)

model.fit(features_2_train, target_2_train)

predicted_2_valid = model.predict(features_2_valid)
score_2 = model.score(features_2_valid, target_2_valid)
RMSE_2 = sqrt(mean_squared_error(target_2_valid, predicted_2_valid))


### Compare predictions and correct answers

In [18]:
model_results = pd.DataFrame({'real_volume'        :[target_0_valid['product'].sum(),
                                                     target_1_valid['product'].sum(),
                                                     target_2_valid['product'].sum()],
                              'predict_volume'      : [predicted_0_valid.sum(),
                                                      predicted_1_valid.sum(),
                                                      predicted_2_valid.sum()],
                              'mean_predict_volume' : [predicted_0_valid.mean(),
                                                      predicted_1_valid.mean(),
                                                      predicted_2_valid.mean()],
                              'RMSE'                : [RMSE_0, RMSE_1, RMSE_2], 
                              'R2-score'            : [score_0, score_1, score_2]
}
)

model_results

,real_volume,predict_volume,mean_predict_volume,RMSE,R2-score
0,2.310666e+06,2.314842e+06,92.593681,37.650563,0.273593
1,1.723502e+06,1.723938e+06,68.957534,0.894446,0.999622
2,2.372809e+06,2.381887e+06,95.275462,40.003681,0.202021


### Intermediate output

1. The model for the 2nd region has the highest R2 metric.
2. Minimum RMSE value obtained for the 2nd region
2. The most promising regions in terms of the average predicted amount of raw materials are the 1st and 3rd regions


## Preparation for revenue calculation

### Create constants for key values

In [19]:
#budget for well development in the region
investment = 10_000_000_000

#revenue per unit of product (1000 barrels)
product_price = 450_000                     
                                
#cost of construction of one well
well_development = investment/200

#minimum supply of raw materials to open a well
well_opening = well_development/product_price

#minimum stock of raw materials to break even the region
region_opening = investment/product_price

print ('Minimum amount of raw material in the well:', round (well_opening, 2), 'unit')
print ('Minimum amount of raw materials in the region:', round (region_opening, 2), 'unit')

mean_region_quantity = round(region_opening / well_opening, 2)
print ('Minimum average quantity of raw materials in a well by region:', mean_region_quantity, 'units')

Minimum amount of raw material in the well: 111.11 unit
Minimum amount of raw materials in the region: 22222.22 unit
Minimum average quantity of raw materials in a well by region: 200.0 units


### Compare the resulting minimum value with the average in each region

In [20]:
if model_results.loc[0,'mean_predict_volume'] >= mean_region_quantity:
     print ('The average amount of minimum raw materials in the mine for the first region is enough')
else:
     print ('The average amount of minimum raw materials in the mine for the first region is not enough')

The average amount of minimum raw materials in the mine for the first region is not enough


In [21]:
if model_results.loc[1,'mean_predict_volume'] >= mean_region_quantity:
     print ('The average amount of minimum raw materials in the mine in the second region is enough')
else:
     print ('The average amount of minimum raw materials in the mine in the second region is not enough')
        

The average amount of minimum raw materials in the mine in the second region is not enough


In [22]:
if model_results.loc[2,'mean_predict_volume'] >= mean_region_quantity:
     print ('The average amount of minimum raw materials in the mine in the third region is enough')
else:
     print ('The average amount of minimum raw materials in the mine in the third region is not enough')
        

The average amount of minimum raw materials in the mine in the third region is not enough


### Summary

- To open a well, it is necessary to have at least 111.11 units of raw materials in it (taking into account that 1 unit of raw materials is 1000 barrels).
- To develop a region, it is necessary that all wells in this region have at least 22,222.22 units of raw materials (assuming that one unit is 1,000 barrels).
- If we compare the average value of the number of units of raw materials with the average minimum required, it will seem that in each of the regions there is not enough raw material, but it is not correct to compare the average in this case, since in one mine there can be a lot, but in many mines it is small. Since 200 mines out of 500 will be used, we will not use the average value.

## Revenue calculation function

### Create a function

In [23]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    oil_sum = selected.sum()
    oil_revenue = oil_sum * product_price - investment
    return pd.DataFrame({'oil_sum':[oil_sum], 'oil_revenue':[oil_revenue]})

- oil_sum - запас сырья
- oil_revenue - прибыль

### Combine predictions and answers into a table

In [24]:
target_0_predict = pd.concat([target_0_valid.reset_index(drop=True), pd.DataFrame(predicted_0_valid)], axis=1)
target_1_predict = pd.concat([target_1_valid.reset_index(drop=True), pd.DataFrame(predicted_1_valid)], axis=1)
target_2_predict = pd.concat([target_2_valid.reset_index(drop=True), pd.DataFrame(predicted_2_valid)], axis=1)

target_0_predict = target_0_predict.rename(columns = {0:'predicted_product'})
target_1_predict = target_1_predict.rename(columns = {0:'predicted_product'})
target_2_predict = target_2_predict.rename(columns = {0:'predicted_product'})

### Select wells with the maximum predicted amount of raw materials

#### First region

In [25]:
max_oil_0 = target_0_predict.sort_values(by='predicted_product', ascending=False)[:200]
quantity_0 = max_oil_0['predicted_product'].sum()
print ('The total amount of raw materials in 200 wells in the first region:', quantity_0)

The total amount of raw materials in 200 wells in the first region: 31124.8931034763


#### Second region

In [26]:
max_oil_1 = target_1_predict.sort_values(by='predicted_product', ascending=False)[:200]
quantity_1 = max_oil_1['predicted_product'].sum()
print ('The total amount of raw materials in 200 wells in the second region:', quantity_1)

The total amount of raw materials in 200 wells in the second region: 27743.00159143885


#### Third region

In [27]:
max_oil_2 = target_2_predict.sort_values(by='predicted_product', ascending=False)[:200]
quantity_2 = max_oil_2['predicted_product'].sum()
print ('The total amount of raw materials in 200 wells in the third region:', quantity_2)

The total amount of raw materials in 200 wells in the third region: 29718.375030064024


### Calculate the recenue for the top 200 wells

#### First region

In [32]:
revenue_0 = revenue (max_oil_0['product'], max_oil_0['predicted_product'], 200)
revenue_0

,oil_sum,oil_revenue
0,29681.923564,3.356866e+09


#### Second region

In [33]:
revenue_1 = revenue (max_oil_1['product'], max_oil_1['predicted_product'], 200)
revenue_1

,oil_sum,oil_revenue
0,27589.081548,2.415087e+09


#### Third region

In [34]:
revenue_2 = revenue (max_oil_2['product'], max_oil_2['predicted_product'], 200)
revenue_2

,oil_sum,oil_revenue
0,27974.781092,2.588651e+09


### Revenue Summary

The maximum profit can be obtained from 200 wells in the first region.

## Revenue and Risk Calculation

### Bootstrap procedure for each region

#### Bootstrap for the first region

In [37]:
region_0_boot = pd.DataFrame()
for i in range(1000):
    target_subsample = target_0_predict['product'].sample(n=500, replace=True, random_state = state)
    probs_subsample = target_0_predict['predicted_product'][target_subsample.index]
    region_0_boot = region_0_boot.append(revenue(target_subsample, probs_subsample, 200), ignore_index=True)

region_0_boot

,oil_sum,oil_revenue
0,22917.426785,3.128421e+08
1,23144.438289,4.149972e+08
2,22808.114110,2.636513e+08
3,22886.090330,2.987406e+08
4,22931.414629,3.191366e+08
...,...,...
995,23252.580704,4.636613e+08
996,23395.908459,5.281588e+08
997,24396.518944,9.784335e+08
998,23133.990643,4.102958e+08


#### Bootstrap for the second region

In [38]:
region_1_boot = pd.DataFrame()
for i in range(1000):
    target_subsample = target_1_predict['product'].sample(n=500, replace=True, random_state = state)
    probs_subsample = target_1_predict['predicted_product'][target_subsample.index]
    region_1_boot = region_1_boot.append(revenue(target_subsample, probs_subsample, 200), ignore_index=True)

region_1_boot

,oil_sum,oil_revenue
0,23569.175443,6.061289e+08
1,23234.540340,4.555432e+08
2,24531.475972,1.039164e+09
3,23753.010505,6.888547e+08
4,24059.033684,8.265652e+08
...,...,...
995,23282.226879,4.770021e+08
996,23618.382422,6.282721e+08
997,23410.634979,5.347857e+08
998,22801.905945,2.608577e+08


#### Bootstrap for the third region

In [39]:
region_2_boot = pd.DataFrame()
for i in range(1000):
    target_subsample = target_2_predict['product'].sample(n=500, replace=True, random_state = state)
    probs_subsample = target_2_predict['predicted_product'][target_subsample.index]
    region_2_boot = region_2_boot.append(revenue(target_subsample, probs_subsample, 200), ignore_index=True)

region_2_boot

,oil_sum,oil_revenue
0,21868.507666,-1.591716e+08
1,22619.974623,1.789886e+08
2,23650.777353,6.428498e+08
3,23349.055369,5.070749e+08
4,24100.518156,8.452332e+08
...,...,...
995,23641.816440,6.388174e+08
996,23151.227183,4.180522e+08
997,22242.200872,8.990392e+06
998,22447.160239,1.012221e+08


### Calculate confidence intervals

#### Confidence interval for the first region

In [40]:
confidence_0 = (region_0_boot.oil_revenue.quantile(0.025), region_0_boot.oil_revenue.quantile(0.975))
confidence_0

(-107438742.09530859, 920160542.1260606)

#### Confidence interval for the second region

In [41]:
confidence_1 = (region_1_boot.oil_revenue.quantile(0.025), region_1_boot.oil_revenue.quantile(0.975))
confidence_1

(106647184.04231901, 941440474.7195092)

#### Confidence interval for the third region

In [42]:
confidence_2 = (region_2_boot.oil_revenue.quantile(0.025), region_2_boot.oil_revenue.quantile(0.975))
confidence_2

(-143499299.1026896, 966169070.351338)

### Calculate development risks for each region

#### Development risk for the first region

In [43]:
limit_0 = len(region_0_boot.query('oil_revenue <= 0'))
risk_0 =  limit_0/len(region_0_boot) * 100
round (risk_0, 2)

5.6

#### Development risk for the second region

In [44]:
limit_1 = len(region_1_boot.query('oil_revenue <= 0'))
risk_1 =  limit_1/len(region_1_boot) * 100
round (risk_1, 2)

0.4

#### Development risk for the third region

In [45]:
limit_2 = len(region_2_boot.query('oil_revenue <= 0'))
risk_2 =  limit_2/len(region_2_boot) * 100
round (risk_2, 2)

7.0

### Bring everything into one table

In [47]:
region = pd.DataFrame({'Mean revenure of each region, rub.'            : [region_0_boot.oil_revenue.mean(),
                                                                     region_1_boot.oil_revenue.mean(),
                                                                     region_2_boot.oil_revenue.mean()],
                             '95% confidence                   '       : [confidence_0,
                                                                     confidence_1,
                                                                     confidence_2],
                             'Loss probability, %'                     : [risk_0,
                                                                     risk_1,
                                                                     risk_2]}, index=['Region 1','Region 2','Region 3'])
region

,"Mean revenure of each region, rub.",95% confidence,"Loss probability, %"
Region 1,4.219455e+08,"(-107438742.09530859, 920160542.1260606)",5.6
Region 2,5.365917e+08,"(106647184.04231901, 941440474.7195092)",0.4
Region 3,4.143944e+08,"(-143499299.1026896, 966169070.351338)",7.0


## General conclusion

1. The most promising regions in terms of the average predicted amount of raw materials are the 1st and 3rd regions
2. To open a well, it is necessary to have at least 111.11 units of raw materials in it (taking into account that 1 unit of raw materials is 1000 barrels).
3. To develop a region, it is necessary that all wells in this region have at least 22,222.22 units of raw materials (assuming that one unit is 1,000 barrels).
4. The maximum profit can be obtained from 200 wells in the first region.
5. The average profit is the highest in the second region.
6. The probability of losses is the lowest in the second region, only 0.4%.

## Region selection

It is recommended to explore the second region, as it is predicted to have the highest profits and the lowest losses.